In [0]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

In [0]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline 

In [0]:
import sys
sys.path.append('/usr/local/lib/python3.6.7/site-packages/')
sys.path.append('/usr/local/lib/python3.7/site-packages/rdkit/')

In [0]:
% cd /usr/local/lib/python3.7/site-packages/

In [0]:
from rdkit import Chem


In [0]:
% cd /content/

In [0]:
%cd /content/rdkit/Contrib/SA_Score/

In [0]:
%%writefile sascorer.py
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
import pickle

import math
from collections import defaultdict

import os.path as op

_fscores = None


def readFragmentScores(name='fpscores'):
    import gzip
    global _fscores
    # generate the full path filename:
    if name == "fpscores":
        name = op.join(op.dirname(__file__), name)
    _fscores = pickle.load(gzip.open('%s.pkl.gz' % name))
    outDict = {}
    for i in _fscores:
        for j in range(1, len(i)):
            outDict[i[j]] = float(i[0])
    _fscores = outDict


def numBridgeheadsAndSpiro(mol, ri=None):
    nSpiro = rdMolDescriptors.CalcNumSpiroAtoms(mol)
    nBridgehead = rdMolDescriptors.CalcNumBridgeheadAtoms(mol)
    return nBridgehead, nSpiro


def calculateScore(m):
    if _fscores is None:
        readFragmentScores()

    # fragment score
    fp = rdMolDescriptors.GetMorganFingerprint(m,
                                               2)  # <- 2 is the *radius* of the circular fingerprint
    fps = fp.GetNonzeroElements()
    score1 = 0.
    nf = 0
    for bitId, v in fps.items():
        nf += v
        sfp = bitId
        score1 += _fscores.get(sfp, -4) * v
    score1 /= nf

    # features score
    nAtoms = m.GetNumAtoms()
    nChiralCenters = len(Chem.FindMolChiralCenters(m, includeUnassigned=True))
    ri = m.GetRingInfo()
    nBridgeheads, nSpiro = numBridgeheadsAndSpiro(m, ri)
    nMacrocycles = 0
    for x in ri.AtomRings():
        if len(x) > 8:
            nMacrocycles += 1

    sizePenalty = nAtoms**1.005 - nAtoms
    stereoPenalty = math.log10(nChiralCenters + 1)
    spiroPenalty = math.log10(nSpiro + 1)
    bridgePenalty = math.log10(nBridgeheads + 1)
    macrocyclePenalty = 0.
    # ---------------------------------------
    # This differs from the paper, which defines:
    #  macrocyclePenalty = math.log10(nMacrocycles+1)
    # This form generates better results when 2 or more macrocycles are present
    if nMacrocycles > 0:
        macrocyclePenalty = math.log10(2)

    score2 = 0. - sizePenalty - stereoPenalty - spiroPenalty - bridgePenalty - macrocyclePenalty

    # correction for the fingerprint density
    # not in the original publication, added in version 1.1
    # to make highly symmetrical molecules easier to synthetise
    score3 = 0.
    if nAtoms > len(fps):
        score3 = math.log(float(nAtoms) / len(fps)) * .5

    sascore = score1 + score2 + score3

    # need to transform "raw" value into scale between 1 and 10
    min = -4.0
    max = 2.5
    sascore = 11. - (sascore - min + 1) / (max - min) * 9.
    # smooth the 10-end
    if sascore > 8.:
        sascore = 8. + math.log(sascore + 1. - 9.)
    if sascore > 10.:
        sascore = 10.0
    elif sascore < 1.:
        sascore = 1.0

    return sascore


def processMols(mols):
    print('smiles\tName\tsa_score')
    for i, m in enumerate(mols):
        if m is None:
            continue

        s = calculateScore(m)

        smiles = Chem.MolToSmiles(m)
        print(smiles + "\t" + m.GetProp('_Name') + "\t%3f" % s)


if __name__ == '__main__':
    import sys
    import time

    t1 = time.time()
    readFragmentScores("fpscores")
    t2 = time.time()

    suppl = Chem.SmilesMolSupplier(sys.argv[1],titleLine=False,nameColumn=-1)
    t3 = time.time()
    processMols(suppl)
    t4 = time.time()

    print('Reading took %.2f seconds. Calculating took %.2f seconds' % ((t2 - t1), (t4 - t3)),
          file=sys.stderr)

In [0]:
%cd /content/

In [0]:
def write_to_file_without_c(file,list_):
  out = open(file, "w")
  for l in list_:
    if l != 'C':
      out.write(l+'\n')
  out.close()
  return


for i in range(1,6):
  lig_gen_file = "protein.seq.decode.results_bs4_"+str(i)+"_larger_ligands_generated"
  lig_gen_1000_file = "protein.seq.decode.results_bs4_1000_"+str(i)+"_larger_ligands_generated"
  lig_train_file = "ligands_train.space_sep_seq_"+str(i)
  out_file = "ligands_train_join.space_sep_seq_"+str(i)
  lig_train = [''.join([value.strip() for value in line.split(' ')]) for line in open(lig_train_file, 'r')]
  lig_gen = [line.strip() for line in open(lig_gen_file, 'r')]
  lig_gen_1000 = [line.strip() for line in open(lig_gen_1000_file, 'r')]
  write_to_file_without_c(lig_gen_file,lig_gen)
  write_to_file_without_c(lig_gen_1000_file,lig_gen_1000)
  write_to_file_without_c(lig_train_file,lig_train)

In [9]:
%cd /content/rdkit/Contrib/SA_Score/


/content/rdkit/Contrib/SA_Score


In [0]:
%%shell
for filename in /content/{ligands_train*,protein*}
do
echo $(basename $filename)
tmp=$(basename $filename)
new_name='/content/sas_'$tmp
python sascorer.py $filename > $new_name
done

In [20]:
%cd /content/

/content


In [0]:
import seaborn as sns

def show_hist_old(list_to_show_gen,list_to_gen_1000, list_to_show_train,plot_title,ds_num):
  plt.figure()
  sns.distplot( list_to_show_gen , label="generated ligands")
  sns.distplot( list_to_show_train , label="ligands used for train")
  sns.distplot( list_to_gen_1000 , label="generated 1000 ligands")
  plt.legend()
  plt.title(plot_title+str(ds_num))
  return



def show_hist(list_to_show_gen,list_to_gen_1000, list_to_show_train,plot_title,ds_num):
  # cache data for future processing
  for data_id, data in enumerate([list_to_show_gen, list_to_gen_1000, list_to_show_train]):
    filename = f'{plot_title}_{ds_num}_{data_id}_sas.txt'
    np.savetxt(filename, data)
    print(f'Saved data to {filename}')
    #files.download(filename)
  
  plt.figure()
  sns.distplot( list_to_show_gen ,  label="generated ligands", kde=False, norm_hist=True)
  sns.distplot( list_to_show_train ,  label="ligands used for train", kde=False, norm_hist=True)
  sns.distplot( list_to_gen_1000 ,  label="generated 1000 ligands", kde=False, norm_hist=True)
  plt.legend()
  plt.title(plot_title+str(ds_num))
  return


def sas_extract(file):
  list_final=[]
  with open(file, 'r') as f:
    next(f)
    for line in f:      
      line=line.strip()
      parts = line.split('\t')
      list_final.append(float(parts[2]))
  return list_final
      
  
  
  

for i in range(1,6):
  print('processing ds '+str(i))
  lig_gen_file = "sas_protein.seq.decode.results_bs4_"+str(i)+"_larger_ligands_generated"
  lig_gen_1000_file = "sas_protein.seq.decode.results_bs4_1000_"+str(i)+"_larger_ligands_generated"
  lig_train_file = "sas_ligands_train.space_sep_seq_"+str(i)
  lig_generated_sas = sas_extract(lig_gen_file)
  lig_generated_1000_sas = sas_extract(lig_gen_1000_file)
  lig_train_sas = sas_extract(lig_train_file)
  
  
  show_hist(lig_generated_sas,lig_generated_1000_sas,lig_train_sas,'sas',i)
  

In [0]:
!rm *.txt

In [0]:
for i in range(1,6):
  lig_gen_file = "protein.seq.decode.results_bs4_1000_"+str(i)+"_larger_ligands_generated"
  lig_train_file = "ligands_train.space_sep_seq_"+str(i)
  out_file = "check_gen_vs_train_1000_"+str(i)
  lig_generated = [line.strip() for line in open(lig_gen_file, 'r')]
  lig_train = [''.join([value.strip() for value in line.split(' ')]) for line in open(lig_train_file, 'r')]
  uniq_lig_train = []
  for ligand in lig_train:
    if ligand not in uniq_lig_train:
      uniq_lig_train.append(ligand)
  
  print(len(lig_train))
  print(len(uniq_lig_train))
  
  
  lig_train=uniq_lig_train
  
  out = open(out_file, "w")

  k=0
  for x in lig_generated:
      for y in lig_train:
          if str(x.strip()) == str(y.strip()):
              #print(y.strip() + ' ' + x.strip() + ' ' + str(k) + '\n')
              out.write(y.strip()+'\n')
      k=k+1
  
  out.close()

In [0]:
%cd /content/

In [0]:
! rm *.txt

In [0]:
from rdkit import Chem
from rdkit.Chem import Lipinski
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen
from rdkit.Chem.Descriptors import qed


import seaborn as sns

plt.style.use('seaborn-pastel')

from google.colab import files


def show_hist(list_to_show_gen,list_to_gen_1000, list_to_show_train,plot_title,ds_num):
  # cache data for future processing
  for data_id, data in enumerate([list_to_show_gen, list_to_gen_1000, list_to_show_train]):
    filename = f'{plot_title}_{ds_num}_{data_id}.txt'
    np.savetxt(filename, data)
    print(f'Saved data to {filename}')
    #files.download(filename)
  
  plt.figure()
  sns.distplot( list_to_show_gen ,  label="generated ligands", kde=False, norm_hist=True)
  sns.distplot( list_to_show_train ,  label="ligands used for train", kde=False, norm_hist=True)
  sns.distplot( list_to_gen_1000 ,  label="generated 1000 ligands", kde=False, norm_hist=True)
  plt.legend()
  plt.title(plot_title+str(ds_num))
  return

In [0]:
for i in range(1,6):
  print('processing ds '+str(i))
  lig_gen_file = "protein.seq.decode.results_bs4_"+str(i)+"_larger_ligands_generated"
  lig_gen_1000_file = "protein.seq.decode.results_bs4_1000_"+str(i)+"_larger_ligands_generated"
  lig_train_file = "ligands_train.space_sep_seq_"+str(i)
  lig_generated = [line.strip() for line in open(lig_gen_file, 'r')]
  lig_generated_1000 = [line.strip() for line in open(lig_gen_1000_file, 'r')]
  lig_train = [''.join([value.strip() for value in line.split(' ')]) for line in open(lig_train_file, 'r')]
  uniq_lig_train = []
  for ligand in lig_train:
    if ligand not in uniq_lig_train:
      uniq_lig_train.append(ligand)
  
  print(len(lig_train))
  print(len(uniq_lig_train))
  lig_train=uniq_lig_train
  
  
  gen_logP = list(map(lambda x: Crippen.MolLogP(Chem.MolFromSmiles(x)), lig_generated))
  gen_qed = list(map(lambda x: qed(Chem.MolFromSmiles(x)), lig_generated))
  gen_h_accept = list(map(lambda x: rdMolDescriptors.CalcNumLipinskiHBA(Chem.MolFromSmiles(x)), lig_generated))
  gen_h_donor = list(map(lambda x: rdMolDescriptors.CalcNumLipinskiHBD(Chem.MolFromSmiles(x)), lig_generated))
  gen_rot_bond = list(map(lambda x: rdMolDescriptors.CalcNumRotatableBonds(Chem.MolFromSmiles(x)), lig_generated))
  gen_mol_weight = list(map(lambda x: Descriptors.MolWt(Chem.MolFromSmiles(x)), lig_generated))
  gen_tpsa = list(map(lambda x: rdMolDescriptors.CalcTPSA(Chem.MolFromSmiles(x)), lig_generated))
  gen_len = list(map(lambda x: len(x), lig_generated))
  
  
  
  
  gen_1000_logP = list(map(lambda x: Crippen.MolLogP(Chem.MolFromSmiles(x)), lig_generated_1000))
  gen_1000_qed = list(map(lambda x: qed(Chem.MolFromSmiles(x)), lig_generated_1000))
  gen_1000_h_accept = list(map(lambda x: rdMolDescriptors.CalcNumLipinskiHBA(Chem.MolFromSmiles(x)), lig_generated_1000))
  gen_1000_h_donor = list(map(lambda x: rdMolDescriptors.CalcNumLipinskiHBD(Chem.MolFromSmiles(x)), lig_generated_1000))
  gen_1000_rot_bond = list(map(lambda x: rdMolDescriptors.CalcNumRotatableBonds(Chem.MolFromSmiles(x)), lig_generated_1000))
  gen_1000_mol_weight = list(map(lambda x: Descriptors.MolWt(Chem.MolFromSmiles(x)), lig_generated_1000))
  gen_1000_tpsa = list(map(lambda x: rdMolDescriptors.CalcTPSA(Chem.MolFromSmiles(x)), lig_generated_1000))
  gen_1000_len = list(map(lambda x: len(x), lig_generated_1000))
  
  
  print('train part')
  
  
  train_logP = list(map(lambda x: Crippen.MolLogP(Chem.MolFromSmiles(x)), lig_train))
  train_qed = list(map(lambda x: qed(Chem.MolFromSmiles(x)), lig_train))
  train_h_accept = list(map(lambda x: rdMolDescriptors.CalcNumLipinskiHBA(Chem.MolFromSmiles(x)), lig_train))
  train_h_donor = list(map(lambda x: rdMolDescriptors.CalcNumLipinskiHBD(Chem.MolFromSmiles(x)), lig_train))
  train_rot_bond = list(map(lambda x: rdMolDescriptors.CalcNumRotatableBonds(Chem.MolFromSmiles(x)), lig_train))
  train_mol_weight = list(map(lambda x: Descriptors.MolWt(Chem.MolFromSmiles(x)), lig_train))
  train_tpsa = list(map(lambda x: rdMolDescriptors.CalcTPSA(Chem.MolFromSmiles(x)), lig_train))
  train_len = list(map(lambda x: len(x), lig_train))
  
  
  print('plotting')
  
  
  show_hist(gen_logP,gen_1000_logP,train_logP,'logP',i)
  show_hist(gen_qed,gen_1000_qed,train_qed,'qed',i)
  show_hist(gen_h_accept,gen_1000_h_accept,train_h_accept,'h_accept',i)
  show_hist(gen_h_donor,gen_1000_h_donor, train_h_donor,'h_donor',i)
  show_hist(gen_rot_bond,gen_1000_rot_bond, train_rot_bond,'rot_bond',i)
  show_hist(gen_mol_weight,gen_1000_mol_weight, train_mol_weight,'mol_weight',i)
  show_hist(gen_tpsa,gen_1000_tpsa,train_tpsa,'tpsa',i)
  show_hist(gen_len,gen_1000_len, train_len,'len',i)
    
  
  
  
  


In [0]:
plot_names = ['h_accept', 'h_donor', 'len', 'logP', 'mol_weight', 'qed', 'rot_bond', 'tpsa']
for name in plot_names:
    for model_id in range(1, 6):
        for data_id in range(3):
            filename = f'{name}_{model_id}_{data_id}.txt'
            files.download(filename)

In [0]:
from rdkit import Chem
from rdkit.Chem import AllChem
db_fingerprints = [AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(m), 2, nBits=1024) for m in lig_train]

In [0]:
def max_tanimoto_compute(ligand_list,db_fingerprints, plot_name, ds_num):
  max_tanimoto=[]
  for query_smiles in ligand_list:
    query_fp = AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(query_smiles), 2, nBits=1024)
    max_score = max([AllChem.DataStructs.FingerprintSimilarity(query_fp, db_fp) for db_fp in db_fingerprints])
    max_tanimoto.append(max_score)
  print(len(max_tanimoto))
  filename = f'{plot_name}_{ds_num}.txt'
  np.savetxt(filename, max_tanimoto)
  #plt.hist(max_tanimoto)
  #plt.show()
  return

In [0]:
from rdkit import Chem
from rdkit.Chem import AllChem

for i in range(1,6):
  lig_gen_file = "protein.seq.decode.results_bs4_"+str(i)+"_larger_ligands_generated"
  lig_gen_1000_file = "protein.seq.decode.results_bs4_1000_"+str(i)+"_larger_ligands_generated"
  lig_train_file = "ligands_train.space_sep_seq_"+str(i)
  lig_generated = [line.strip() for line in open(lig_gen_file, 'r')]
  lig_generated_1000 = [line.strip() for line in open(lig_gen_1000_file, 'r')]
  lig_train = [''.join([value.strip() for value in line.split(' ')]) for line in open(lig_train_file, 'r')]
  uniq_lig_train = []
  for ligand in lig_train:
    if ligand not in uniq_lig_train:
      uniq_lig_train.append(ligand)
  
  print(len(lig_train))
  print(len(uniq_lig_train))
  lig_train=uniq_lig_train
  
  db_fingerprints_ = [AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(m), 2, nBits=1024) for m in lig_train]
  
  max_tanimoto_compute(ligand_list=lig_generated_1000,db_fingerprints=db_fingerprints_, plot_name='generated_1000', ds_num=i)
  max_tanimoto_compute(ligand_list=lig_generated,db_fingerprints=db_fingerprints_, plot_name='generated', ds_num=i)

In [0]:
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold

for i in range(1,6):
  lig_gen_file = "protein.seq.decode.results_bs4_"+str(i)+"_larger_ligands_generated"
  lig_gen_1000_file = "protein.seq.decode.results_bs4_1000_"+str(i)+"_larger_ligands_generated"
  lig_train_file = "ligands_train.space_sep_seq_"+str(i)
  lig_generated = [line.strip() for line in open(lig_gen_file, 'r')]
  lig_generated_1000 = [line.strip() for line in open(lig_gen_1000_file, 'r')]
  lig_train = [''.join([value.strip() for value in line.split(' ')]) for line in open(lig_train_file, 'r')]
  uniq_lig_train = []
  for ligand in lig_train:
    if ligand not in uniq_lig_train:
      uniq_lig_train.append(ligand)
  
  print(len(lig_train))
  print(len(uniq_lig_train))
  lig_train=uniq_lig_train
  
  gen_murcko = list(map(lambda x: MurckoScaffold.MurckoScaffoldSmiles(x), lig_generated))
  gen_1000_murcko = list(map(lambda x: MurckoScaffold.MurckoScaffoldSmiles(x), lig_generated_1000))
  train_murcko = list(map(lambda x: MurckoScaffold.MurckoScaffoldSmiles(x), lig_train))

  print(i)
  print('number of unique scsffolds in generated set ',len(set(gen_murcko)))
  print('number of unique scsffolds in generated 1000 set ',len(set(gen_1000_murcko)))
  print('number of unique scsffolds in train set ',len(set(train_murcko)))
  print('number of common scsffolds ',len(list(set(gen_murcko).intersection(train_murcko))))
  print('number of common scsffolds ',len(list(set(gen_1000_murcko).intersection(train_murcko))))
  